# FFT baby


In [ ]:
%matplotlib qt5
import numpy as np
from matplotlib import pyplot as plt

from typing import Callable

In [ ]:
def Z_pow(N: int, sign: int = 1) -> Callable[[int], complex]:
    Z = np.exp(2j*np.pi / N)
    memo = {}
    def _Z_pow(nk: int):
        Znk = memo.get(nk)
        if Znk is None:
            Znk = Z**nk
            memo[nk] = Znk
        return Znk
    # precalculate
    for i in range(N):
        for j in range(N):
            _Z_pow(sign*i*j)
    return _Z_pow

def DFT(yn: np.ndarray) -> np.ndarray:
    "O(n^2)"
    N = yn.size
    Z = Z_pow(N)
    Yn = np.empty(N, dtype=np.complex128)
    for n in range(N):
        Ynk = 0
        for k in range(N):
            Ynk += Z(n*k) * yn[k]
        Yn[n] = Ynk
    Yn *= (2*np.pi)**-0.5
    return Yn

def IDFT(Yn: np.ndarray) -> np.ndarray:
    "O(n^2)"
    N = Yn.size
    Z = Z_pow(N, sign=-1)
    ynk = np.empty((N, N), dtype=np.complex128)
    for n in range(N):
        for k in range(N):
            ynk[n, k] = Z(-n*k) * Yn[k]
    yn = np.sum(ynk, axis=1)
    yn *= (2*np.pi)**0.5 / N
    return yn

In [ ]:
def _f(x):
    if x < 1:
        return x
    return x - 2
f = np.vectorize(_f)

f = lambda x: np.sin(np.pi * x)

x = np.linspace(0, 2, 100)
y = f(x)
print(y.shape)
Yn = DFT(y)
print(Yn.shape)
yn = IDFT(Yn)
print(yn.shape)

plt.figure()
plt.scatter(x, y)
plt.scatter(x, yn.real)
plt.legend(["y", "IDFT(DFT(y))"])
plt.show()